<a href="https://colab.research.google.com/github/tiensu/Coding-The-Deep-Learning-Revolution/blob/master/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import datetime as dt
import os
from tensorflow import keras

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [0]:
base_path = 'D:\\MACHINE_LEARNING\\EBOOKs\\CODING_THE_DEEP_LEARNING_REVOLUTION\\PRACTICE\\Tensorboard\\'

In [0]:
def get_batch(x_data, y_data, batch_size):
    idxs = np.random.randint(0, len(y_data), batch_size)
    return x_data[idxs,:,:], y_data[idxs]

In [0]:
class ConvNet(object):
    def __init__(self):
        self._model_def()
        
    def _model_def(self):
        # create placeholder for input variables
        self.input_images = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
        # scale input data
        input = tf.div(self.input_images, 255.0)
        # create place holder for label
        self.labels = tf.placeholder(tf.int64, shape=[None, 1])
        # convert the label to one hot value
        y_one_hot = tf.reshape(tf.one_hot(self.labels, 10), [-1, 10])
        
        # create some convolutional layers
        layer1 = self._create_new_conv_layer(input, 1, 32, [5, 5], [2, 2], [1, 1], [2, 2], name='layer1')
        layer2 = self._create_new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], [1, 1], [2, 2], name='layer2')
        
        # flatten the output ready for the fully connected output stage - after two layers of stride 2 pooling, we go
        # from 28x28, to 14x14 to 7x7 (64 channels). To create the fully connected, "dense" layer, the new shape need 
        # to be [-1, 7*7*64]. However,bellow is a generalized method of determining the flattened shape.
        shape = layer2.get_shape().as_list()
        flattened_size = shape[1]*shape[2]*shape[3]
        flattened = tf.reshape(layer2, shape=[-1, flattened_size])
        
        initializer = tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=False)
        self.prob = tf.placeholder_with_default(1.0, shape=())
        with tf.name_scope("first_dense"):
            # setup some weights and bias values for this layer, then activate with ReLU
            wd1 = tf.Variable(initializer((flattened_size, 300)), name='wd1')
            bd1 = tf.Variable(tf.zeros(300), name='bd1')
            dense_layer1 = tf.matmul(flattened, wd1) + bd1
            dense_layer1 = tf.nn.relu(dense_layer1)
            dense_layer1 = tf.nn.dropout(dense_layer1, self.prob)
        with tf.name_scope("second_dense"):
            # another layer with softmax activations
            wd2 = tf.Variable(initializer((300, 10)), name='wd2')
            bd2 = tf.Variable(tf.zeros(10), name='bd2')
            logits = tf.matmul(dense_layer1, wd2) + bd2
            
        # define loss function
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_one_hot))
        # add the loss to the summary
        tf.summary.scalar('loss', self.loss)
        
        # add an optimiser
        self.optimizer = tf.train.AdamOptimizer().minimize(self.loss)
        
        # get the accuracy and log
        self.accuracy = self._compute_accuracy(logits, y_one_hot)
        tf.summary.scalar('acc', self.accuracy)
        
        # collect the summaries
        self.merged = tf.summary.merge_all()
        
        # setup the initialisation operator
        self.init_op = tf.global_variables_initializer()
        
    def _create_new_conv_layer(self, input, input_channels, output_channels, window_size,
                               pool_size, filt_stride, pool_stride, name):
        initializer = tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=False)
        with tf.name_scope(name):
            # setup the filter input shape for tf.nn.conv_2d
            conv_filt_shape = [window_size[0], window_size[1], input_channels, output_channels]
            # initialise weights and bias for the filter
            weights = tf.Variable(initializer(conv_filt_shape), name='W')
            bias = tf.Variable(tf.zeros((output_channels)), name='b')
            # setup the convolutional layer operation
            filt_stride = [1, filt_stride[0], filt_stride[1], 1]
            out_layer = tf.nn.conv2d(input, weights, filt_stride, padding='SAME')
            # add the bias
            out_layer += bias
            # apply a ReLU non-linear activation
            out_layer = tf.nn.relu(out_layer)
            pool_shape = [1, pool_size[0], pool_size[1], 1]
            pool_strides = [1, pool_stride[0], pool_stride[1], 1]
            out_layer = tf.nn.max_pool(out_layer, ksize=pool_shape, strides=pool_strides, padding='SAME')
        return out_layer
    
    def _compute_accuracy(self, logits, labels):
        prediction = tf.argmax(logits, 1)
        equality = tf.equal(prediction, tf.argmax(labels, 1))
        accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
        return accuracy

In [0]:
def train_model(model, batch_size, epochs):
    with tf.Session() as sess:
        sess.run(model.init_op)
        train_writer = tf.summary.FileWriter(base_path +
                                             "{}/".format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")),
                                             sess.graph)
        for i in range(epochs):
            image_batch, label_batch = get_batch(x_train, y_train, batch_size)
            loss, _, acc = sess.run([model.loss, model.optimizer, model.accuracy],
                                    feed_dict={model.input_images: image_batch.reshape(-1, 28, 28, 1),
                                               model.labels: label_batch.reshape(-1, 1),
                                               model.prob: 0.5})
            if i % 50 == 0:
                print("Iteration {} of {} - loss: {:.3f}, training accuracy: {:.2f}%".
                      format(i, epochs, loss, acc*100))
                summary = sess.run(model.merged, feed_dict={model.input_images: image_batch.reshape(-1, 28, 28, 1),
                                                    model.labels: label_batch.reshape(-1, 1)})
                train_writer.add_summary(summary, i)
        acc = sess.run(model.accuracy, feed_dict={model.input_images: x_test.reshape(-1, 28, 28, 1),
                                                    model.labels: y_test.reshape(-1, 1)})
        print("Final test accuracy is {:.2f}%".format(acc * 100))

In [0]:
if __name__ == "__main__":
    model = ConvNet()
    train_model(model, 32, 5000)

Iteration 0 of 5000 - loss: 2.268, training accuracy: 15.62%
Iteration 50 of 5000 - loss: 0.667, training accuracy: 78.12%
Iteration 100 of 5000 - loss: 0.499, training accuracy: 81.25%
Iteration 150 of 5000 - loss: 0.134, training accuracy: 96.88%
Iteration 200 of 5000 - loss: 0.277, training accuracy: 87.50%
Iteration 250 of 5000 - loss: 0.102, training accuracy: 96.88%
Iteration 300 of 5000 - loss: 0.099, training accuracy: 96.88%
Iteration 350 of 5000 - loss: 0.090, training accuracy: 96.88%
Iteration 400 of 5000 - loss: 0.415, training accuracy: 93.75%
Iteration 450 of 5000 - loss: 0.330, training accuracy: 87.50%
Iteration 500 of 5000 - loss: 0.204, training accuracy: 93.75%
Iteration 550 of 5000 - loss: 0.048, training accuracy: 100.00%
Iteration 600 of 5000 - loss: 0.004, training accuracy: 100.00%
Iteration 650 of 5000 - loss: 0.115, training accuracy: 93.75%
Iteration 700 of 5000 - loss: 0.163, training accuracy: 93.75%
Iteration 750 of 5000 - loss: 0.183, training accuracy: 